In [426]:
from sklearn import tree
from math import log
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
import random

In [403]:
#loading data
df = pd.read_csv('tennis.csv')

In [404]:
df

,Day,outlook,temp,humidity,windy,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [405]:
#preprocessing
lb = LabelEncoder() 
df['play_'] = lb.fit_transform(df['play'] ) 
Y = df.iloc[:,-1]

In [406]:
X = df.values
X=np.delete(X, np.s_[0:1], axis=1)
X=np.delete(X, np.s_[-2:], axis=1)
X

array([['Sunny', 'Hot', 'High', 'Weak'],
       ['Sunny', 'Hot', 'High', 'Strong'],
       ['Overcast', 'Hot', 'High', 'Weak'],
       ['Rain', 'Mild', 'High', 'Weak'],
       ['Rain', 'Cool', 'Normal', 'Weak'],
       ['Rain', 'Cool', 'Normal', 'Strong'],
       ['Overcast', 'Cool', 'Normal', 'Strong'],
       ['Sunny', 'Mild', 'High', 'Weak'],
       ['Sunny', 'Cool', 'Normal', 'Weak'],
       ['Rain', 'Mild', 'Normal', 'Weak'],
       ['Sunny', 'Mild', 'Normal', 'Strong'],
       ['Overcast', 'Mild', 'High', 'Strong'],
       ['Overcast', 'Hot', 'Normal', 'Weak'],
       ['Rain', 'Mild', 'High', 'Strong']], dtype=object)

In [407]:
Y

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: play_, dtype: int64

In [408]:
target=list(Y)
examples=X
a=list(df.columns)
a=a[1:-2]
attributes=a

In [409]:
examples

array([['Sunny', 'Hot', 'High', 'Weak'],
       ['Sunny', 'Hot', 'High', 'Strong'],
       ['Overcast', 'Hot', 'High', 'Weak'],
       ['Rain', 'Mild', 'High', 'Weak'],
       ['Rain', 'Cool', 'Normal', 'Weak'],
       ['Rain', 'Cool', 'Normal', 'Strong'],
       ['Overcast', 'Cool', 'Normal', 'Strong'],
       ['Sunny', 'Mild', 'High', 'Weak'],
       ['Sunny', 'Cool', 'Normal', 'Weak'],
       ['Rain', 'Mild', 'Normal', 'Weak'],
       ['Sunny', 'Mild', 'Normal', 'Strong'],
       ['Overcast', 'Mild', 'High', 'Strong'],
       ['Overcast', 'Hot', 'Normal', 'Weak'],
       ['Rain', 'Mild', 'High', 'Strong']], dtype=object)

In [410]:
np.array([target]).transpose()

array([[0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [411]:
attributes


['outlook', 'temp', 'humidity', 'windy']

In [412]:
def entropy(target):
    target=list(target)
    p_list=[]
    l=len(target)
    for i in list(set(target)):
        p_list.append(target.count(i)/l)
    if 1 in p_list:
        return 0
    S=0
    for i in p_list:
        if i!=0:
            S+=i*log(i,2)
    return -S

def info_gain(examples,attributes,A,target):
    S=entropy(target)
    l=len(target)
    sm=0
    tmp=np.array(examples)
    values_A=list(set(tmp[:,attributes.index(A)]))
    for v in values_A:
        S_v=[]
        for i in range(len(target)):
            if examples[i][attributes.index(A)]==v:
                S_v.append(target[i])
        sm+=len(S_v)*entropy(S_v)/l
    i_g=S-sm
    return i_g


In [413]:
def findRoot(examples, target, attributes):
    ig_max=-1
    root=0
    for A in attributes:
        if info_gain(examples,attributes,A,target)>ig_max:
            ig_max=info_gain(examples, attributes, A, target)
            root=A
    return root

def DecisionTreeClassifier(examples, target, attributes):
    tree_={}
    root=findRoot(examples, target, attributes)

    tree_[root]={}
    if 0 not in target:
        tree_[root]=1
        return tree_
    if 1 not in target:
        tree_[root]=0
        return tree_
    if len(attributes)==0:
        if sum(target)>=len(target)/2:
            tree_[root]=1
            return tree_
        else:
            tree_[root]=0
            return tree_
    examples=np.array(examples)
    values_root=list(set(examples[:,attributes.index(root)]))
    branch={}
    tree_[root]=branch    
    for v in values_root:
        branch[v]={}
        examp_new=[]
        target_new=[]    
        for i in range(len(target)):
            if examples[i][attributes.index(root)]==v:
                examp_new.append(examples[i])
                target_new.append(target[i])
        if examp_new==[]:
            if sum(target)>=len(target)/2:
                branch[v]=1
            else:
                branch[v]=0
        else:
            tmp=[z for z in attributes]
            tmp.remove(root)
            examp_new=np.delete(examp_new, np.s_[attributes.index(root):attributes.index(root)+1], axis=1)
            branch[v]=DecisionTreeClassifier(examp_new, target_new, tmp)
    attributes.remove(root)
    return tree_
            

In [414]:
examples_=examples
target_=target
attributes_=attributes
tree_=DecisionTreeClassifier(examples_, target_, attributes_)

## Obtained Trained Decision Tree Model

In [415]:
tree_ 

{'outlook': {'Overcast': {'temp': 1},
  'Rain': {'windy': {'Strong': {'temp': 0}, 'Weak': {'temp': 1}}},
  'Sunny': {'humidity': {'High': {'temp': 0}, 'Normal': {'temp': 1}}}}}

In [354]:
#show the tree

In [ ]:
def test_model(trained_model, test_examples, attributes):
    for i in test_examples:
    
    
    return


## Testing IRIS dataset

In [625]:
iris = datasets.load_iris()

In [627]:
examples_iris=iris['data']
target_iris=iris['target']
target_name=iris['target_names']
attributes_iris=iris['feature_names']

In [630]:
target_name

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [631]:
attributes_iris

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [632]:
examples_iris.shape

(150, 4)

In [633]:
split=int(0.7*len(target_iris))       #splitting into 70:30
train_examples=examples_iris[:split]
train_target=target_iris[:split]
test_examples=examples_iris[split:]
test_target=target_iris[split:]

xz_=np.c_[train_examples,np.array([train_target]).transpose()]
xz=np.random.permutation(xz_)
train_examples=np.delete(xz, np.s_[-1:], axis=1)
train_target=np.uint8(xz[:,-1])

In [634]:
train_examples.shape

(105, 4)

In [635]:
train_examples

array([[5. , 3.4, 1.5, 0.2],
       [5.8, 2.6, 4. , 1.2],
       [5.7, 2.6, 3.5, 1. ],
       [4.9, 2.4, 3.3, 1. ],
       [5.5, 2.3, 4. , 1.3],
       [5. , 2.3, 3.3, 1. ],
       [5.2, 4.1, 1.5, 0.1],
       [5.4, 3. , 4.5, 1.5],
       [6. , 3.4, 4.5, 1.6],
       [5.1, 3.5, 1.4, 0.3],
       [6.1, 3. , 4.6, 1.4],
       [7. , 3.2, 4.7, 1.4],
       [5.1, 3.4, 1.5, 0.2],
       [5.4, 3.7, 1.5, 0.2],
       [6.5, 3. , 5.8, 2.2],
       [4.4, 3.2, 1.3, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [4.4, 3. , 1.3, 0.2],
       [4.6, 3.6, 1. , 0.2],
       [5. , 3.2, 1.2, 0.2],
       [5.5, 3.5, 1.3, 0.2],
       [5.5, 2.4, 3.8, 1.1],
       [5.5, 4.2, 1.4, 0.2],
       [6.1, 2.8, 4. , 1.3],
       [5.4, 3.9, 1.7, 0.4],
       [5.6, 3. , 4.5, 1.5],
       [5. , 3.6, 1.4, 0.2],
       [6.3, 3.3, 6. , 2.5],
       [5.8, 2.7, 5.1, 1.9],
       [6.2, 2.9, 4.3, 1.3],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.5, 1.3, 0.3],
       [6.6, 3. , 4.4, 1.4],
       [5.7, 2.8, 4.5, 1.3],
       [4.5, 2

In [636]:
train_target

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 2, 0, 1, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1], dtype=uint8)

In [637]:
def modify_examples(train_examples, split_values):
    train_examples=[list(i) for i in train_examples]
    for i in range(len(train_examples)):
        for j in range(len(train_examples[0])):
            if train_examples[i][j]>split_values[j][2]:
                train_examples[i][j]=">" + str(split_values[j][2])
            elif train_examples[i][j]>split_values[j][1]:
                train_examples[i][j]=">"+str(split_values[j][1])
            elif train_examples[i][j]>split_values[j][0]:
                train_examples[i][j]=">"+str(split_values[j][1])
            else:
                train_examples[i][j]="<"+str(split_values[j][0])
    return train_examples

    
change_point=[]
for i in range(len(train_target)):
    try:
        if train_target[i]!=train_target[i+1]:
            change_point.append(i)
    except:
        pass
tmp_data=[]
for i in change_point:
    tmp_data.append((np.array(train_examples[i])+np.array(train_examples[i+1]))/2)
tmp_data=np.array(tmp_data)
tmp_data=np.around(tmp_data,decimals=3)
split_values={}
for i in range(len(attributes_iris)):
    tmp_=tmp_data[:,i]
    tmp_.sort()
    tmp_=list(set(tmp_))
    mid=int(len(tmp_)/2)
    if mid>5:
        tmp_=[tmp_[0],tmp_[mid],tmp_[-1]]
    split_values[i]=tmp_

train_examples=modify_examples(train_examples, split_values)
train_examples
            

[['<6.05', '>3.05', '<2.5', '<0.55'],
 ['>5.55', '<2.75', '>3.65', '>0.95'],
 ['>5.55', '<2.75', '>5.15', '>0.95'],
 ['<6.05', '<2.75', '>5.15', '>0.95'],
 ['<6.05', '<2.75', '>3.65', '>1.2'],
 ['<6.05', '<2.75', '>5.15', '>0.95'],
 ['<6.05', '>3.55', '<2.5', '<0.55'],
 ['<6.05', '>3.05', '>3.65', '>1.2'],
 ['>5.55', '>3.05', '>3.65', '>1.2'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['<6.05', '>3.55', '<2.5', '<0.55'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['<6.05', '>3.55', '<2.5', '<0.55'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['<6.05', '>3.05', '<2.5', '<0.55'],
 ['<6.05', '<2.75', '>3.65', '>0.95'],
 ['<6.05', '>3.55', '<2.5', '<0.55'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['<6.05', '>3.55', '<2.5', '<0.55'],
 ['>5.55', '>3.05', '>3.65', '>1.2'],
 ['<6.05', 

In [639]:
tree_iris=DecisionTreeClassifier(train_examples, train_target, attributes_iris)

In [640]:
tree_iris     #obtained trained decision tree model

{'petal length (cm)': {'<2.5': {'sepal length (cm)': 0},
  '>3.65': {'sepal length (cm)': 1},
  '>5.15': {'sepal length (cm)': 1}}}

In [641]:
#testing model
test_examples=modify_examples(test_examples, split_values)
test_examples

[['>6.0', '>3.05', '>3.65', '>1.2'],
 ['<6.05', '<2.75', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '<2.75', '>3.65', '>1.2'],
 ['>6.0', '>3.55', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '<2.75', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>5.55', '<2.75', '>3.65', '>1.2'],
 ['>5.55', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.55', '>3.65', '>1.2'],
 ['>6.0', '<2.75', '>3.65', '>1.2'],
 ['>5.55', '<2.75', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>5.55', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '<2.75', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.05', '>3.65', '>1.2'],
 ['>6.0', '>3.55', '>3.65', '>1.2

In [ ]:
def test_data(trained_model, row, attributes):
    root=tree_iris.keys[0]
    if row[attributes_iris.key(root)]
    